In [1]:
import numpy as np

###########
# constants
###########

n =  10 # number of stacs
m_stac =  1# mass of individual stac

R_z_dot = # tbd, this is our stack velocity in the z direction
R_z = # array of vertical positions of each stack

dt = 1

w_motor =  # motor ang velocity (radians per second)
l_screw =  # screw lead (meters)
r_screw =  # screw radius (meters)
v =  (w_motor * l_screw) / (2 * np.pi)  # linear velocity of screw shaft (meters per second)


MOI_cube =  #3x3 matrix for moment of inertia of cubesat without stacs
MOI_stac = #3x3 matrix giving moment of inertia of an individual stac about it's own axis
n = #number of stacs left in the cubesat
positions = #nx3 length v giving the x y z locations of the stacs (will need to code up function for this in future)
m_stac = #mass of stac 

##################
# helper functions
##################

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@ double check correctness
def v_shaft(w_motor,l_screw):
    return (w_motor * l_screw) / (2 * np.pi)

# note that R_z_dot 
def make_J_dot(R_x, R_y, R_z_dot, n):
    xz = -n * m_stac * R_x * R_z_dot
    yz = -n * m_stac * R_y * R_z_dot
    return np.matrix([[0,  0,  xz],
                      [0,  0,  yz],
                      [xz, yz, 0]])

#find moment of inertia of cubesat

def inertia_of_cubesat(MOI_cube,MOI_stac,n,positions):
    
    MOI_mat = np.empty([3,3])
    
    for i in range(3):
        for j in range(3):
            for x in range(n):
                if i != j:
                    g = 0
                else:
                    g = positions[x,i] * positions[x,j]
            ab += g  
            
        MOI_mat[i,j] = MOI_cube[i,j] + n * MOI_stac[i,j] + m_stac*ab
        
    return MOI_mat
            
    






SyntaxError: invalid syntax (<ipython-input-1-0d63f8e61a77>, line 7)

In [ ]:
#driving torque required - checking if it below stepper motor 

linear_thrust = # linear thrust required to drive PCBsats
efficiency = #efficiency leadscrew

driving_torque = (l_screw * linear_thrust)/(2 * np.pi * efficiency)




### TBD
* update stac vertical positions at each time step
* compute inertial matrix based on positions
* encode some way of adjusting inertial matrix when a stac is released (if z > threshold: DECREASE N BY ONE??)
* 
* NEW PLAN
* use `inertia_of_cubesat` function to instantiate initial inertia matrix
* gather the vertical positions of the terms in `positions`, our nx3 matrix of the pcb positions (impleentation can either pick out all the `z` terms and save in new matrix since these are the only things we care about, or we can just reuse `positions`)
* update `z` values of `positions` after some time step, `dt`.
*      if any positions are updated beyond threshold (`z > threshold`) remove them. I THINK WE CAN JUST DECREASE N BY COUNT of BELOW THRESHOLD
*      recompute inertia matrix on update `positions`
*      store result. I don't think we need to save ALL of the intermediate `positions`, just the most recent one

In [ ]:
# ========================
# dynamics
# ========================

# Use paul's function inertia_of_cubesat to initialize inertial matrix

I = inertia_of_cubesat(MOI_cube,MOI_stac,n,positions) # current inertial matrix

# NOTE: the following variables are lists, appended with
# calculations at each time step. should start with initial val
v_stac = [initial]
angVel = [initial]
                      
# CONSTANTS
# dt defined above
# w_motor :: ang vel provided by motor, acting on leadscrew
# torqueCube :: torque from coil, reaction wheel, whatever we decide on
# THRESHOLD :: defines vertical position at which pcb has left the cubesat

def update_dynamics(w_motor, torqueCube, dt):
    global n
    global I
    
    # computes/updates new stac velocity
    v = v_shaft(w_motor, l_screw)
    v_stac.append(v)
    
    # z_dot = vertical displacement of pcb's
    z_dot = v * dt
    
    # update positions
    count = 0
    for row in range(n):
        # new_z = z + z_dot
        new_z = positions[row][2] + z_dot
        
        # update positions
        positions[row][2] = new_z
        
        # count number pcb's which left cubesat
        if new_z >= THRESHOLD:
            count += 1      
            
    # decrease number of pcbs in satellite by `count`
    n -= count
    
    # compute new inertia matrix with updated positions + number of pcb's
    I = inertia_of_cubesat(MOI_cube, MOI_stac, n, positions)
    
    # computes/updates new angular velocity
    angAccel = np.linalg.inv(I[-2]) * torqueCube
    angVel.append(angVel[-1] + angAccel * dt)
    
# ========================
# simulation
# ========================

# fills in I, v_stac, angVel lists at each time step in (start, end)

start = 
end = 

for i in np.arrange(start, end+dt, dt):
    update_dynamics(num_stac_curr, torqueMotor, torqueCube, dt)